In [ ]:
# 주피터
import os

os.chdir('G:/내 드라이브/projects/NLP-StockMarket/model_fin/')

In [ ]:
# 코랩
from google.colab import drive
drive.mount("/content/drive")

import os
path = '/content/drive/My Drive/projects/NLP-StockMarket/model_fin/' # 본인 구글 드라이브 계정마다 살짝씩 다를 수도 있음
os.chdir(path)

Mounted at /content/drive


## 필요모듈 import

In [ ]:
!pip install konlpy   # 코랩에서만 실행

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 453 kB 34.7 MB/s 


In [ ]:
# 모듈 정리 필요
from tqdm import tqdm
import scipy as sp
import pandas as pd
import numpy as np

from konlpy.tag import Kkma    
from konlpy.tag import Hannanum    
from konlpy.tag import Okt        
from konlpy.tag import *
import nltk
import pickle

# from wordcloud import WordCloud, STOPWORDS
from PIL import Image

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz
from sklearn.tree import export_graphviz
from datetime import datetime, timedelta

import matplotlib.font_manager as fm
plt.rc('font', family='NanumGothic')

import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False

import warnings
warnings.filterwarnings('ignore')

In [ ]:
kkma = Kkma()
hannanum = Hannanum()

## 자연어 데이터 import(날짜 가져오기 위해 먼저 가져옴)

In [ ]:
# LG화학, 삼성SDI, SK이노베이션, 고려아연, 포스코케미칼
stock_name = '삼성SDI'

In [ ]:
naver_news = pd.read_csv('./data/refined_naver_news.csv', index_col=0)
daum_news = pd.read_csv('./data/refined_daum_news.csv', index_col=0)
naver_talks = pd.read_csv(f'./data/refined_naver_talks_{stock_name}.csv', index_col=0)
daum_talks = pd.read_csv(f'./data/refined_daum_talks_{stock_name}.csv', index_col=0)
youtube = pd.read_csv(f'./data/refined_youtube_{stock_name}.csv', index_col=0)

In [ ]:
naver_talks['Date'] = pd.to_datetime(naver_talks['Date'])
daum_talks['Date'] = pd.to_datetime(daum_talks['Date'])

In [ ]:
news_df = pd.concat([naver_news, daum_news, naver_talks, daum_talks ,youtube])
news_df['Date'] = pd.to_datetime(news_df['Date'].astype(str))     # str(변수) 는 자체 내장함수, 변수 하나씩밖에 적용 안됨.
news_df.sort_values('Date', ignore_index=True, inplace=True)

,Date,Title
0,2021-01-01,서학개미 사로잡은 美 ESG ETF 올해도 高高할까
1,2021-01-01,목표가 속속 100만원대 흥 오른 삼성바이오로직스
2,2021-01-01,신생 PEF운용사 안타 떡볶이업체 SJ코레 사들여 1년만에 매각 80억 차익
3,2021-01-01,삼성전자 신고가 랠리 9만전자 가능할까
4,2021-01-01,작년 매출 신기록 쓴 카카오 엔씨 올해 더 좋다


In [ ]:
news_df = news_df[news_df[news_df['Date']== '2021-01-04'].index[0] : ]
news_df.head(2)

## 주가 데이터

### KRX에서 받은 csv로 가져오기

In [ ]:
stock_df = pd.read_csv(f'./data/{stock_name}_주가_데이터.csv', usecols = ['일자','등락률'])
stock_df['일자'] = pd.to_datetime(stock_df['일자'])
stock_df.head()

,일자,등락률
0,2021-01-04,6.85
1,2021-01-05,2.24
2,2021-01-06,-0.87
3,2021-01-07,2.65
4,2021-01-08,5.87


In [ ]:
start = str(stock_df.iloc[0, 0])
end = str(stock_df.iloc[-1, 0])
print(start)
print(end)

2021-01-04 00:00:00
2022-06-30 00:00:00


### 감성지수 계산

In [ ]:
stock_df['감성지수'] = 0
for i in tqdm(range(len(stock_df))):
    if stock_df['등락률'][i] > 0:
        stock_df['감성지수'][i] = 1
    else:
        stock_df['감성지수'][i] = 0
stock_df.head()

100%|██████████| 369/369 [00:00<00:00, 414.65it/s]


,일자,등락률,감성지수
0,2021-01-04,6.85,1
1,2021-01-05,2.24,1
2,2021-01-06,-0.87,0
3,2021-01-07,2.65,1
4,2021-01-08,5.87,1


### news_df 날짜 하루빠르게 변환

In [ ]:
## 뉴스일자 조정(예측대상(주가)의 일자와 맞추기 위해)
news_df['일자'] = news_df['Date'] + timedelta(days=1)

### news_df와 stock_df 합치기

In [ ]:
df = news_df.merge(stock_df)
df.columns = [df.columns[0], df.columns[1], '주가의 날짜', '등락률(y)', '감성지수']
df.drop_duplicates('Title', inplace = True, ignore_index = True)  # 기사제목 중복 제거
print(len(df))
df.head()

440380


,Date,Title,주가의 날짜,등락률(y),감성지수
0,2021-01-03,年50조 몰고오는 개미들 동학개미 시즌2 예고,2021-01-04,6.85,1
1,2021-01-03,메리츠證 지금은 대세상승 초입 2023년까지 구조적 상승장,2021-01-04,6.85,1
2,2021-01-03,1000 2000 위기때 버팀목은 실적장세,2021-01-04,6.85,1
3,2021-01-03,표 주간 주요 증시 지표,2021-01-04,6.85,1
4,2021-01-03,부고 이종길 미래에셋자산운용 상무 씨 모친상,2021-01-04,6.85,1


## 예측 모델 적용

In [ ]:
# 형태소 분석을 위한 함수
def tokenizer(text):
    okt = Okt()
    return okt.morphs(text)

In [ ]:
# X : title, y : price
def data_split(X, y):
    # 수집한 데이터 읽어오기
    # news_df = pd.read_excel()
    # 학습셋, 테스트셋 분리
    X_list = X.tolist()
    y_list = y.tolist()
    # 데이터의 80%는 학습셋, 20%는 테스트셋
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    return x_train, x_test, y_train, y_test

In [ ]:
x_train, x_test, y_train, y_test = data_split(df['Title'], df['감성지수'])

In [ ]:
vect = CountVectorizer()

In [ ]:
# tfidf
tfidf = TfidfTransformer()
# tfidf = TfidfVectorizer(lowercase=False, tokenizer=tokenizer)

In [ ]:
# 로지스틱
logistic = LogisticRegression(random_state=0)    
# logistic = LogisticRegression(C=2, penalty='l2', random_state=0)     # C의 숫자가 너무 크면 과적합 (기본 1), penalty로 과적합 방지

In [ ]:
pipeline = Pipeline([('vect',vect), ('tfidf',tfidf), ('clf',logistic)], verbose=True)

In [ ]:
pipeline.fit(x_train, y_train)

[Pipeline] .............. (step 1 of 3) Processing vect, total=   2.9s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.1s
[Pipeline] ............... (step 3 of 3) Processing clf, total=   3.9s


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(random_state=0))],
         verbose=True)

In [ ]:
y_pred = pipeline.predict(x_test)

In [ ]:
print('모델 정확도:', accuracy_score(y_test, y_pred))

모델 정확도: 0.5978957988037761


In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(pipeline, x_train, y_train, cv=3, scoring='f1_micro')
print('평균 모델 score:', scores.mean())

[Pipeline] .............. (step 1 of 3) Processing vect, total=   2.1s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.1s
[Pipeline] ............... (step 3 of 3) Processing clf, total=   2.6s
[Pipeline] .............. (step 1 of 3) Processing vect, total=   1.9s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.1s
[Pipeline] ............... (step 3 of 3) Processing clf, total=   2.9s
[Pipeline] .............. (step 1 of 3) Processing vect, total=   2.1s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.1s
[Pipeline] ............... (step 3 of 3) Processing clf, total=   3.9s
평균 모델 score: 0.5877036529927157


In [ ]:
#  모델 저장
def save_model(model):
    with open('pipe.dat', 'wb') as fp:     # 쓰기, 바탕화면에 저장됨
        pickle.dump(model, fp)
    print('저장완료')     # 학습된 모델 저장 완료

In [ ]:
save_model(pipeline)

저장완료


In [ ]:
# 모델 사용 함수
def model_prediction():  
    # 객체를 복원, 저장된 모델 불러오기
    with open('pipe.dat','rb') as fp:     # 읽기
        pipe = pickle.load(fp)
    while True :
        text = input('뉴스 타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) : ')
        example = [text]
        # 예측 정확도
        r1 = np.max(pipe.predict_proba(example)*100)     # 확률값을 구해서 *100
        # 예측 결과
        r2 = pipe.predict(example)[0]     # 긍정('1'), 부정('0')
        if text == 'q':
            print("예측을 종료합니다.")
            break
        if r2 == '1':
            print(f'{stock_name} 주가가 상승할 것으로 예상됩니다.')
        else: 
            print(f'{stock_name} 주가가 하락할 것으로 예상됩니다.')
        print('확률 : %.3f' % r1)
        print('------------------------------------------------')

In [ ]:
model_prediction()

뉴스 타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) : [속보] 코스피, 21.77포인트(0.95%) 오른 2322.11 출발
삼성SDI 주가가 하락할 것으로 예상됩니다.
확률 : 56.602
------------------------------------------------
뉴스 타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) : 코스피 상승 출발…2,320대로 올라
삼성SDI 주가가 하락할 것으로 예상됩니다.
확률 : 53.070
------------------------------------------------
뉴스 타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) : LG에너지솔루션, 美배터리 공장 재검토 악재 아냐-현대차증권
삼성SDI 주가가 하락할 것으로 예상됩니다.
확률 : 77.195
------------------------------------------------
뉴스 타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) : 글람, 실적 성장 등 체질 개선 "올해 매출 400억·영업익 100억 전망"
삼성SDI 주가가 하락할 것으로 예상됩니다.
확률 : 57.988
------------------------------------------------
뉴스 타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) : 삼성SDI 영업이익 상승 전망
삼성SDI 주가가 하락할 것으로 예상됩니다.
확률 : 53.681
------------------------------------------------
뉴스 타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) : q
예측을 종료합니다.
